In [1]:
import numpy as np
import cvxpy as cp
from matplotlib import pyplot as plt

from time import time
from functools import partial

import sys
sys.path.append('../../')

from Environment import Quadrotor1D
from Subroutines import QuadrotorEst,ApproxDAP,max_norm,find_stable_radius,SafeTransit,spectral_radius
from time import time
from Controllers import SafeDAP
from scipy.linalg import sqrtm
import pickle as pkl
%load_ext autoreload
%autoreload 2

In [2]:
class TimeStepsReached(Exception):
    pass
def evaluate(space_dim,dt,K_m_dt,l_m_dt):
    O = np.zeros((space_dim,space_dim))
    I = np.eye(space_dim)

    A = np.vstack([np.hstack([O,dt*I]),
                   np.hstack([-K_m_dt*I,-l_m_dt*I])]) + np.eye(2*space_dim)

    return  np.max(np.abs(np.linalg.eigvals(A)))

# Set up the system

In [3]:
m = 1
K_stab = np.array([[2/3,1]])
l = 0.25
# l = 0.0

dt = 1
w_max = 0.2

env=Quadrotor1D(m,K_stab,l,w_max,dt)

# $\Theta_{ini}$

In [4]:
alpha_limit = (0.5,1.2)
beta_limit = (0.2,0.4)

# Optimization Parameters

In [5]:
# Constants
x_dim = env.AK.shape[-1]
u_dim = env.B.shape[-1]


I_x = np.eye(x_dim)  
I_u = np.eye(u_dim)  

w_cov = np.eye(x_dim) * env.w_max * 1/12 # Assume uniform distribution.

# Cost matrices

Q = np.array([[1,0],
                [0,1]])*1
R = np.eye(u_dim)*1

# Constraints

x_max = 5
x_min = -1

v_max = 2
v_min = -2

u_max = 5
u_min = -9.8

D_x = np.vstack([I_x,-I_x]) # Constraints
d_x = np.array([x_max,v_max,-x_min,-v_min])

D_u = np.vstack([I_u,-I_u])
d_u = np.array([u_max,-u_min])

# Lookback lengths
H=10
H_benchmark = 40

M0 = np.zeros((H,u_dim,x_dim))

e_x = 0 # Usually e_x is not zero 
e_u = 0 # For safe DAP with known B, e_u = 0

refit_per_step = 50
pre_run_steps = 2
TD_steps  = 30 # The least number of steps taken in Phase 1.

# eta_bars = [0.1,0.5,1.0,1.5,2.0]
eta_bars = [0.1]

# Experiments

In [6]:

def PlainSim(timesteps,n_trials):
    
    def main_loop():
        env=Quadrotor1D(m,K_stab,l,w_max,dt)
        
        x_hist = []
        u_hist = []
        # main loop
        for _ in range(timesteps):
            x = env.state()
            x_hist.append(x)
            
            u = 0.1*(np.random.rand()-0.5) 
            u_hist.append(u)
            env.step(u)
            
            if _%100 == 0:
                print('Step',_)

        data = {}
        data['x'] = x_hist
        data['u'] = u_hist
        return data
    
    trial_data = []
    _ = 0 
    while _ < n_trials: 
        t = time()
        print('Trial {}'.format(_))
        trial_data.append(main_loop())
        _+=1

        print('Time for trial:',time()-t)
    return trial_data 

In [7]:
n_trials = 1
timesteps = 400
      

trial_data = PlainSim(timesteps,n_trials)


Trial 0
Step 0
Step 100
Step 200
Step 300
Time for trial: 0.010774612426757812


In [8]:
est = QuadrotorEst(np.array([[2/3,1]]),dt,alpha_limit,beta_limit)

In [9]:
x_hist = np.array(trial_data[0]['x']).reshape(-1,x_dim)
u_hist = np.array(trial_data[0]['u'][:-1]).reshape(-1,u_dim)

In [10]:
est.est(x_hist,u_hist)

15.83637764137229 [[1.         1.        ]
 [0.         0.64641295]]


(array([[1.        , 1.        ],
        [0.        , 0.64641295]]),
 array([[0.       ],
        [0.9059553]]),
 0.12262786789699316)

In [11]:
alpha = cp.Variable(nonneg = True )
beta = cp.Variable(nonneg = True)

x_t = x_hist[:-1,:]
x_t_1 = x_hist[1:,:]

A_hat = cp.vstack([cp.hstack([1,dt]),cp.hstack([0,1-beta])])
B_hat = cp.vstack([0,alpha])

objective = cp.Minimize(cp.sum_squares(x_t_1.T - ((A_hat-B_hat @ K_stab) @ x_t.T + B_hat @ u_hist.T )))

prob = cp.Problem(objective)

prob.solve()

15.83637764137229